# Import thư viện

In [20]:
import pandas as pd
import os
import glob
import csv

# Map tên tỉnh

In [ ]:
province_map = {
    "hanoi": "Hà Nội",
    "haiphong": "Hải Phòng",
    "danang": "Đà Nẵng",
    "hochiminh": "Hồ Chí Minh",
    "cantho": "Cần Thơ",
    "angiang": "An Giang",
    "bacgiang": "Bắc Giang",
    "backan": "Bắc Kạn",
    "baclieu": "Bạc Liêu",
    "bacninh": "Bắc Ninh",

    "baria-vungtau": "Bà Rịa - Vũng Tàu",
    "bentre": "Bến Tre",
    "binhdinh": "Bình Định",
    "binhduong": "Bình Dương",
    "binhphuoc": "Bình Phước",
    "binhthuan": "Bình Thuận",
    "camau": "Cà Mau",
    "caobang": "Cao Bằng",
    "daklak": "Đắk Lắk",
    "daknong": "Đắk Nông",

    "dienbien": "Điện Biên",
    "dongnai": "Đồng Nai",
    "dongthap": "Đồng Tháp",
    "gialai": "Gia Lai",
    "hagiang": "Hà Giang",
    "hanam": "Hà Nam",
    "hatinh": "Hà Tĩnh",
    "haiduong": "Hải Dương",
    "haugiang": "Hậu Giang",
    "hoabinh": "Hòa Bình",

    "hue": "Thừa Thiên Huế",
    "hungyen": "Hưng Yên",
    "khanhhoa": "Khánh Hòa",
    "kiengiang": "Kiên Giang",
    "kontum": "Kon Tum",
    "laichau": "Lai Châu",
    "lamdong": "Lâm Đồng",
    "langson": "Lạng Sơn",
    "laocai": "Lào Cai",
    "longan": "Long An",
    "namdinh": "Nam Định",

    "nghean": "Nghệ An",
    "ninhbinh": "Ninh Bình",
    "ninhthuan": "Ninh Thuận",
    "phutho": "Phú Thọ",
    "phuyen": "Phú Yên",
    "quangbinh": "Quảng Bình",
    "quangnam": "Quảng Nam",
    "quangngai": "Quảng Ngãi",
    "quangninh": "Quảng Ninh",
    "quangtri": "Quảng Trị",

    "soctrang": "Sóc Trăng",
    "sonla": "Sơn La",
    "tayninh": "Tây Ninh",
    "thaibinh": "Thái Bình",
    "thainguyen": "Thái Nguyên",
    "thanhhoa": "Thanh Hóa",
    "tiengiang": "Tiền Giang",
    "travinh": "Trà Vinh",
    "tuyenquang": "Tuyên Quang",
    "vinhlong": "Vĩnh Long",
    "vinhphuc": "Vĩnh Phúc",
    "yenbai": "Yên Bái"
}

# Tiền xử lý dữ liệu
* Đọc dữ liệu các tỉnh từ folder hotels_data
* Thêm cột Province
* Xử lý giá trị NaN
* Xuất file khách sạn 63 tỉnh

In [ ]:
folder_path = 'hotels_data'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

all_data = []

for file in csv_files:
    # Lấy tên tỉnh từ tên file
    filename = os.path.basename(file)
    province_key = os.path.splitext(filename)[0].split('_')[0]
    province = province_map.get(province_key, province_key)

    df = pd.read_csv(file)

    # Thêm cột Province
    df['Province'] = province

    # Xử lý NaN:

    # 1. Overview Price: mean
    df['Overview Price'] = (
        df['Overview Price']
        .str.replace('VND', '', regex=False)
        .str.replace('.', '', regex=False)
        .str.strip()
    )
    df['Overview Price'] = pd.to_numeric(df['Overview Price'], errors='coerce')

    mean_price = df['Overview Price'].mean()

    df['Overview Price'].fillna(mean_price, inplace=True)
    df['Overview Price'] = df['Overview Price'].apply(lambda x: f"{x:,.0f} VND".replace(",", "."))

    # 2. Các điểm đánh giá: mean
    for col in ['Overall Rating', 'Staff', 'Facilities', 'Cleanliness', 'Comfort', 'Value for Money', 'Location', 'Free Wifi']:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False) #chuyển thành dấu chấm để nhận diện số
        df[col] = pd.to_numeric(df[col], errors='coerce') #chuyển sang số
        mean_score = df[col].mean()

        mean_score = round(mean_score, 1)
        mean_score = int(mean_score)
        df[col].fillna(mean_score, inplace=True)

    # 3. Checkin/Checkout Time: mode
    for col in ['Checkin Time', 'Checkout Time']:
        mode_time = df[col].mode()[0]
        df[col].fillna(mode_time, inplace=True)

    # 4. Popular Facilities: mode
    if 'Popular Facilities' in df.columns:
        mode_facilities = df['Popular Facilities'].mode()[0]
        df['Popular Facilities'].fillna(mode_facilities, inplace=True)

    all_data.append(df)


In [42]:
# Gộp tất cả vào một dataframe lớn
combined_df = pd.concat(all_data, ignore_index=True)
combined_df.to_csv('hotels_data_final.csv', index=False, encoding='utf-8-sig')